In [38]:
#Final Project Script  
#Agentic AI for financial analysis 
#MODEL/CODE INSPO: https://python.langchain.com/docs/tutorials/agents/

In [ ]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite
%pip install -qU langchain-xai
%pip install yfinance
%pip install langchain-xai

In [51]:
#import packages
from langgraph.checkpoint.memory import MemorySaver
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage
from langchain_tavily import TavilySearch
from datetime import datetime, timedelta 
import yfinance as yf
import pandas as pd
import numpy as np
import getpass
import os

In [54]:
#For all API keys feel free to slack me! 
#If I post them here GitHub auto-revokes them due to privacy concerns :/ 

In [24]:
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_API_KEY'] = getpass.getpass()
os.environ['TAVILY_API_KEY'] = getpass.getpass()
os.environ['XAI_API_KEY'] = getpass.getpass('Enter API key for xAI: ')

In [ ]:
#create all necessary tools for agent below: 

In [53]:
#create Tavily search tool
search = TavilySearch(max_results=2)

In [40]:
#define tools
tools = [search]

In [41]:
#define agent model for xai llm integration
model = init_chat_model('grok-2',model_provider='xai')

In [42]:
#create memory
memory = MemorySaver()

In [47]:
#create a prompt for the agent role in our project 
agentrole = SystemMessage(content='You are an expert financial analysis agent. Always start by asking for a stock symbol if none is given by the user. Once a symbol is given, list the company represented by the stock and ask for confirmation before proceeding.')

In [48]:
#create the agent 
financeanalyst = create_react_agent(model,tools,prompt=agentrole,checkpointer=memory)
config = {'configurable': {'thread_id': 'testingproj1'}}

In [50]:
#start agent comms 
intro = input('Enter your name to activate the finance agent: ')
for step in financeanalyst.stream(
    {"messages": [('user', intro)]}, config, stream_mode='values'
):
    step['messages'][-1].pretty_print()
stocksymbol = input('Enter symbol here: $')
for step in financeanalyst.stream(
    {"messages": [('user', stocksymbol)]}, config, stream_mode='values'
):
    step['messages'][-1].pretty_print()
confirmation = input('Enter YES if yes, Enter NO if no: ')
for step in financeanalyst.stream(
    {"messages": [('user', confirmation)]}, config, stream_mode='values'
):
    step['messages'][-1].pretty_print()

================================ Human Message =================================

Jack
================================== Ai Message ==================================

Hello Jack! To proceed with the financial analysis of CLPT (ClearPoint Neuro, Inc.), please confirm if this is the correct company. If you have another company in mind, please provide the stock symbol or the company name.
================================ Human Message =================================

CLPT
================================== Ai Message ==================================

The stock symbol CLPT represents ClearPoint Neuro, Inc. Is this the company you would like to analyze?
================================ Human Message =================================

YES
================================== Ai Message ==================================

I am retrieving the latest financial data for ClearPoint Neuro, Inc. (CLPT).
Tool Calls:
  tavily_search (call_92252751)
 Call ID: call_92252751
  Args:
    query: Clear